<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/pydantic_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental"

In [3]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [5]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
# define pydantic classes
class pUser(BaseModel):
  name: str
  age: int
  email: str
p1 = pUser(name="Jane", age=12, email="james.madison@examplepetstore.com")
print(p1)

class Class(BaseModel):
  students: List[pUser]
c1 = Class(students = [pUser(name = "stu1", age=12, email="stu1@example.com"), pUser(name = "stu2", age=13, email="stu2@example.com")])
print(c1)

name='Jane' age=12 email='james.madison@examplepetstore.com'
students=[pUser(name='stu1', age=12, email='stu1@example.com'), pUser(name='stu2', age=13, email='stu2@example.com')]


In [8]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
class WeatherSearch(BaseModel):
  """call this with an airport code to get the weather at the airport"""
  airport_code: str = Field(description="airport code to get weather for")
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'call this with an airport code to get the weather at the airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [10]:
message = "What is the weather in PVG today?"
#llm.invoke(message, functions = [weather_function])
llm_with_function = llm.bind(functions = [weather_function])
llm_with_function.invoke(message)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code": "PVG"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--728cc68d-7d23-4e2d-9eb1-43e184efe91a-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'PVG'}, 'id': 'efde9c12-3782-47f3-b09c-4bc373fce38c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 17, 'total_tokens': 140, 'input_token_details': {'cache_read': 0}})

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}")
])
chain = prompt | llm_with_function
chain.invoke({"input": "What is the weather in PVG today?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code": "PVG"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--924bd228-e4a8-406a-a28a-babe2c38ff1c-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'PVG'}, 'id': '19466299-289c-4044-b424-88664f6907ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 148, 'input_token_details': {'cache_read': 0}})